In [ ]:
%load_ext tensorboard 

In [78]:
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import nltk # just for tokenization
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import tensorflow as tf

In [79]:
import datetime

In [12]:
log = ""

In [278]:
def precision(y_true, y_pred, num_classes):
    # Initialize arrays to store true positives, false positives, and precision
    TP = np.zeros(num_classes)
    FP = np.zeros(num_classes)
    precision_scores = np.zeros(num_classes)

    # Calculate true positives and false positives for each class
    for i in range(num_classes):
        TP[i] = np.sum((y_true == i) & (y_pred == i))
        FP[i] = np.sum((y_true != i) & (y_pred == i))

    # Compute precision for each class
    for i in range(num_classes):
        if TP[i] + FP[i] > 0:
            precision_scores[i] = TP[i] / (TP[i] + FP[i])

    return np.mean(precision_scores)

In [280]:
def hamming_loss(y_true, y_pred):
    # Calculate number of mismatches
    num_mismatches = np.sum(y_true != y_pred)

    # Compute Hamming Loss
    hamming_loss = num_mismatches / (y_true.shape[0] * y_true.shape[1])

    return hamming_loss

In [281]:
def top3_accuracy(predicted_probs, true_labels):

    sorted_indices = np.argsort(predicted_probs, axis=1)[:, ::-1]

    # Check if true labels are in top-3 predicted labels
    top3_correct = np.any(true_labels[np.arange(len(true_labels))[:, None], sorted_indices[:, :3]], axis=1)
    # Calculate top-3 accuracy
    top3_accuracy = np.mean(top3_correct)
    
    return top3_accuracy

In [269]:
class NeuralNetwork:
    
    def __init__(self, raw_data, embeddings, hidden_neurons, test_df, test_embeddings):
        
        self.raw_data = raw_data
        self.random_state = 42
        
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.labels = ['Joy', 'Trust', 'Fear', 'Surprise','Sadness', 'Disgust', 'Anger', 'Anticipation']
        self.emotions_onehot = np.array(raw_data.loc[:, self.labels])
        self.X_train, self.y_train = embeddings, self.emotions_onehot
        self.X_test, self.y_test = test_embeddings, np.array(test_df.loc[:, self.labels])
        # self.__pre_process(embeddings)
        
        self.n_classes = self.y_train.shape[1]
        self.n_input_features = self.X_train.shape[1]
        self.n_hidden_neurons = hidden_neurons
        
        
        #weights from input layer to hidden layer1
        np.random.seed(self.random_state)
        self.W01 = np.random.randn(self.n_input_features, self.n_hidden_neurons)
        self.W12 = np.random.randn(self.n_hidden_neurons, self.n_classes)
        
        self.b01 = np.zeros((1, self.n_hidden_neurons))
        self.b12 = np.zeros((1, self.n_classes))
        
    def __activation(self, activation_function, X):
        if activation_function == "sigmoid":
            return 1/(1+np.exp(-X))
        elif activation_function == "relu":
            return (X > 0) * X
        elif activation_function == "tanh":
            return (np.exp(X) + np.exp(-X))/(np.exp(X) - np.exp(-X))
        
    def __activation_derivative(self, activation_function, X):
        if activation_function == "sigmoid":
            return self.__activation(activation_function, X) * (1 - self.__activation(activation_function, X))
        elif activation_function == "relu":
            return X > 0
        elif activation_function == "tanh":
            return 1 - self.__activation(activation_function, X)**2
    
    def __error(self, preds, ground, error="mean"):
        return -np.mean(ground * np.log(preds) + (1 - ground) * np.log(1 - preds))
        # return 0.5 * preds.shape[1] * ((ground - preds)**2).sum()
        
    
    #Note: output activation will always be sigmoid
    def train(self, epochs=100, lr = 1e-1, hidden_layer_activation = "relu", batch_size = 16, thresold = 0.6):
        log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        summary_writer = tf.summary.create_file_writer(log_folder)
       
        with summary_writer.as_default():
            tf.summary.text("Hidden neuron", str(self.n_hidden_neurons), step=0)
            tf.summary.text("Batch size", str(batch_size), step=0)
            tf.summary.text("Epochs", str(epochs), step=0)
            tf.summary.text("Learning rate", str(lr), step=0)
            tf.summary.text("Hidden Layer Activation", hidden_layer_activation, step=0)

        # forward the data, and then calculate the training accuracy
        self.hidden_layer_activation = hidden_layer_activation
        print(f"number of batches {self.X_train.shape[0]//batch_size}")
        train_error = 0
        for epoch in range(epochs):
            #batch gd
            batches = (self.X_train.shape[0] % batch_size)
            exact_batches = True if batches == 0 else False
            n_batches = (self.X_train.shape[0]//batch_size) if exact_batches else (self.X_train.shape[0]//batch_size + 1)
            for batch in range(n_batches):
                b = batch*batch_size
                b_1 = self.X_train.shape[0] if (not exact_batches) and (batch == n_batches-1) else (batch+1)*batch_size
                self.X_batch = self.X_train[b:b_1]
                self.Y_batch = self.y_train[b:b_1]
                self.Z01 = self.X_batch.dot(self.W01) + self.b01
                self.A01 = self.__activation(hidden_layer_activation, self.Z01)
                self.Z02 = self.A01.dot(self.W12) + self.b12
                self.A02 = self.__activation("sigmoid", self.Z02)
                
                train_error = self.__error(self.A02, self.Y_batch)

                self.backward()

                self.W12 -= lr * self.A01.T.dot(self.d_error_W12)
                self.b12 -= lr * np.sum(self.d_error_W12, axis=0, keepdims = True)
                self.W01 -= lr * self.X_batch.T.dot(self.d_error_W01)

            # print(f"Error {error}")
            if epoch % 10 == 0:
                test_error, precision_metric, hamming_loss_metric, top3_metric = self.test(epoch)

                with summary_writer.as_default():
                    tf.summary.scalar("Top 3 accuracy", top3_metric, step=epoch)
                    tf.summary.scalar("Hamming Loss", hamming_loss_metric, step=epoch)
                    tf.summary.scalar("Precision", precision_metric, step=epoch)
                    tf.summary.scalar("Loss/Test", test_error, step=epoch)
                    tf.summary.scalar("Loss/Train", train_error, step=epoch)
                print(f"Epoch {epoch}, Train error {train_error}, Test error {test_error}, Precision {precision_metric}, top3metric {top3_metric}, Hamming loss {hamming_loss_metric}")
                # train_accuracy = self.accuracy(self.A02, self.Y_batch)
                # print(self.Y_batch, b, b_1)

                
        # log += f"{train_accuracy},"
    def backward(self):
        self.d_error_A02 = (self.A02 - self.Y_batch)/len(self.Y_batch)
        self.d_error_W12 = (self.d_error_A02) * self.__activation_derivative("sigmoid", self.Z02)
        
        self.d_error_W01 = (
            (self.d_error_W12).dot(self.W12.T) * self.__activation_derivative(self.hidden_layer_activation, self.Z01))
    
    
    # def __pre_process(self, embeddings, train_test_ratio = 0.3):
    #     # self.raw_data = self.raw_data.drop("Id",axis=1)
    #     # species_np = np.array(self.raw_data["Species"])
    #     # onehotencoder  = OneHotEncoder(sparse_output = False)
    #     # target_onehot = onehotencoder.fit_transform(species_np.reshape(-1,1))
    #     # self.raw_data=self.raw_data.drop("Species", axis=1)
        
    #     X = embeddings 
    #     y = self.emotions_onehot
        
    #     self.X_train, self.X_test, self.y_train, self.y_test = X, X, y, y
  

    def accuracy(self, y_pred, y_ground):
        y_train_predicted_classes = np.argmax(y_pred, axis = 1)
        y_train_ground_classes = np.argmax(y_ground, axis=1)
        accuracy = ((y_train_predicted_classes == y_train_ground_classes).sum())/len(y_train_predicted_classes)
        return accuracy
               
    def test(self, epoch):
        X, y = self.X_test, self.y_test
        Z01 = X.dot(self.W01) + self.b01
        A01 = self.__activation(self.hidden_layer_activation, Z01)
        Z02 = A01.dot(self.W12) + self.b12
        A02 = self.__activation("sigmoid", Z02)
        predictions = A02.round()
        error = self.__error(A02, y)
        precision_metric = precision(y, predictions, y.shape[1])
        hamming_loss_metric = hamming_loss(y, predictions)
        top3_metric = top3_accuracy(A02, y)
        return error, precision_metric, hamming_loss_metric, top3_metric


    def predict(self, X):
        Z01 = X.dot(self.W01) + self.b01
        A01 = self.__activation(self.hidden_layer_activation, Z01)
        Z02 = A01.dot(self.W12) + self.b12
        A02 = self.__activation("sigmoid", Z02)
        return A02

        
    def print_shapes(self):
        print(f"Xtrain shape {self.X_train.shape}")
        print(f"ytrain shape {self.y_train.shape}")
        print(f"Xtest shape {self.X_test.shape}")
        print(f"ytest shape {self.y_test.shape}")
    
        

# Preprocessing

In [210]:
test_data = pd.read_csv("../../data/EdmondsDance.csv")
train_data = pd.read_csv("../../data/train.csv")
train_data = train_data.rename(columns={"lyrics":"Lyrics"})

In [211]:
#remove unwanted columns
test_data.pop("Unnamed: 0")
test_data.pop("Unnamed: 11")

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
519   NaN
520   NaN
521   NaN
522   NaN
523   NaN
Name: Unnamed: 11, Length: 524, dtype: float64

In [212]:
test_data.head()

,Song,Artists,Lyrics,Joy,Trust,Fear,Surprise,Sadness,Disgust,Anger,Anticipation
0,Apollo,"Hardwell, Amba Shepherd",Just one day in the life<br>So I can understan...,1,1,0,1,0,0,0,0
1,Lullaby,"R3HAB, Mike Williams","Hypnotized, this love out of me<br>Without you...",0,0,1,0,1,0,0,0
2,Melody (Tip Of My Tongue),Mike Williams,I stand a little too close<br>You stare a litt...,1,1,0,0,0,0,0,1
3,Take Me Home,"Cash Cash, Bebe Rexha",I'm falling to pieces<br>But I need this<br>Ye...,0,0,0,1,1,1,0,0
4,City of Dreams,"Dirty South, Alesso","Everything seems like a city of dreams,<br>I n...",0,0,0,1,1,0,0,0


In [213]:
raw_data.describe()

,Joy,Trust,Fear,Surprise,Sadness,Disgust,Anger,Anticipation
count,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000
mean,0.438931,0.561069,0.196565,0.129771,0.353053,0.219466,0.137405,0.475191
std,0.496731,0.496731,0.397780,0.336372,0.478376,0.414280,0.344603,0.499861
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 200
    """
    import gensim.downloader as api
    wv_from_bin = api.load("word2vec-google-news-300")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin
wv_from_bin = load_embedding_model()
# wv_from_bin = load_embedding_model()

In [ ]:
def tokenize(lyric: str) -> list[str]:
    # lowercase the text, remove stop words, punctuation and keep only the words
    lyric.replace("<br>", "\n")
    tokens = nltk.tokenize.word_tokenize(lyric.lower())
    stop_words = stopwords.words("english") + list(string.punctuation)
    lemmatizer = WordNetLemmatizer()
    alpha_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]

    return alpha_tokens

In [ ]:
def vectorise(lyrics: str) -> np.ndarray:
    tokens = tokenize(lyrics)
    lyric_vector = np.zeros(300)
    for token in tokens:
        try:
            lyric_vector += wv_from_bin.get_vector(token.lower())
        except:
            continue
    return lyric_vector / np.linalg.norm(lyric_vector)

In [215]:
# go through each lyrics, tokenize it, vectorize each word, then combine all of them into single average vector and store it in the list

def get_embeddings(raw_data):
    lyrics = raw_data["Lyrics"]
    lyrics_embeddings = []
    unsupported_tokens = set()
    label_embedding_map = {} # dict{str: np.array([])}
    for lyric in tqdm(lyrics):
        lyric_vector = np.zeros(300)
        for token in tokenize(lyric):
            try:
                lyric_vector += wv_from_bin.get_vector(token.lower())
            except KeyError as e:
                # if the word is not present in the glove then key error is raised, so handle the exception and move on
                unsupported_tokens.add(token)
                continue
        lyrics_embeddings.append(lyric_vector)


    lyrics_embeddings = np.stack(lyrics_embeddings)
    scaled_lyrics_embeddings = lyrics_embeddings / np.linalg.norm(lyrics_embeddings, axis=1, keepdims=True)
    return scaled_lyrics_embeddings

In [216]:
train_embeddings = get_embeddings(train_data)
test_embeddings = get_embeddings(test_data)

100%|██████████| 1753/1753 [00:06<00:00, 250.65it/s]


# Train

In [270]:
#  raw_data, embeddings, hidden_neurons, test_df, test_embeddings
nn = NeuralNetwork(train_data, train_embeddings, hidden_neurons = 32, test_df = test_data, test_embeddings = test_embeddings)

In [271]:
nn.train(epochs=1000, lr=1e-2, hidden_layer_activation="relu", batch_size=16)

number of batches 109
Epoch 0, Train error 0.841289734872073, Test error 1.4004723838201174, Precision 0.1220323499695907, top3metric 0.7251908396946565, Hamming loss 0.47638358778625955
Epoch 10, Train error 0.7111049222524194, Test error 1.3633648740214201, Precision 0.13152355603626373, top3metric 0.7366412213740458, Hamming loss 0.37333015267175573
Epoch 20, Train error 0.6759011354175272, Test error 1.4055191513710896, Precision 0.12949696336145294, top3metric 0.7232824427480916, Hamming loss 0.3857347328244275
Epoch 30, Train error 0.6113661662405083, Test error 1.41248228831901, Precision 0.12827800724568716, top3metric 0.7175572519083969, Hamming loss 0.3916984732824427
Epoch 40, Train error 0.5354974573558602, Test error 1.4029939163891672, Precision 0.12687555922637483, top3metric 0.7080152671755725, Hamming loss 0.3974236641221374
Epoch 50, Train error 0.46028956928076453, Test error 1.3891544899082826, Precision 0.12561382686744305, top3metric 0.7022900763358778, Hamming lo

In [282]:
test_error, precision_score, hamming_score, top3_accuracy = nn.test(1) #error, precision, hamming loss, top3 accuracy
print(f"Test error {test_error}, Precision {precision_score}, top3metric {top3_accuracy}, Hamming loss {hamming_score}")

Test error 0.8004127657235502, Precision 0.14349717189735994, top3metric 0.7958015267175572, Hamming loss 0.34899809160305345


In [316]:
song = """
La, la la la la la, la la la
La, la la la la la, la la la
La, la la la la la, la la la
La, la la la la la, la la la
Hold on to me
Don't let me go
Who cares what they see?
Who cares what they know?
Your first name is Free
Last name is Dom
We choose to believe
In where we're from
Man's red flower
It's in every living thing
Mind, use your power
Spirit, use your wings
Freedom!
Freedom!
Freedom!
Freedom
Freedom
Freedom
Hold on to me
Don't let me go
Cheetahs need to eat
Run, antelope
Your first name is King
Last name is Dom
'Cause you still believe
In everyone
When a baby first breathes
When night sees sunrise
When the whale hunts the sea
When man recognizes
Freedom!
Freedom!
Freedom!
Freedom
Freedom
Breathe in
We are from heat
The electric one
Does it shock you to see
He left us the sun?
Atoms in the air
Organisms in the sea
Sun and yes, night
Are made of the same things
Freedom!
Freedom!
Freedom!
Freedom
Freedom
Freedom
Freedom
Freedom
"""

In [317]:
def predict(lyrics: str) -> str:
    song_vector = vectorise(lyrics)[None,:]
    return nn.predict(song_vector)

In [318]:
probs = predict(song)

In [319]:
probs

array([[0.72160929, 0.10660481, 0.36424165, 0.20913482, 0.13555264,
        0.02070832, 0.44212887, 0.26779786]])

In [320]:
np.array(nn.labels)[np.argsort(probs[0])[::-1]]

array(['Joy', 'Anger', 'Fear', 'Anticipation', 'Surprise', 'Sadness',
       'Trust', 'Disgust'], dtype='<U12')

In [291]:
import pickle

with open("../embeddings/nn.pickle", "wb") as f:
    pickle.dump(nn, f)

In [138]:
import pickle


with open("../embeddings/nn.pickle", "rb") as f:
    a = pickle.load(f)

# Generating labels for spotify dataset